# Semantic shifts

In [1]:
from ipynb.fs.full.koselleck import *

## Generating data

### Local neighborhood measure

In [2]:
w='station'

YBIN_NBR=20
YMIN_NBR=1700
YMAX_NBR=1900

In [3]:
# dfnbr=nbr_word(w,ybin=YBIN_NBR,ymin=YMIN_NBR,ymax=YMAX_NBR,force=False,num_proc=4).reset_index()
# dfnbr['period_len']=[int(x[-4:]) - int(x[:4]) for x in dfnbr.period]
# dfnbr.period_len.value_counts()

In [4]:
dfcdist=cdist_word(w,ybin=5,num_proc=4)
dfcdist

[Koselleck] (11:35:03) Finished running cdist_word(station) (+6.6s)


abandon  abandoned  abandoning  abandonment  \
word_   period_   run_                                                 
station 1700-1705 1     0.948871   0.918744    0.955560          NaN   
                  2     1.092415   0.950319    0.966035          NaN   
                  3     1.097515   1.052195    0.854973          NaN   
                  4     0.918367   0.842233    0.937062          NaN   
                  5     1.039314   0.966896    0.908350          NaN   
...                          ...        ...         ...          ...   
        1895-1900 6     1.301627   0.880815    1.116048     1.026179   
                  7     1.017823   1.078721    0.835061     1.007609   
                  8     1.052353   0.993460    0.925477     0.783007   
                  9     1.078712   1.081699    0.855514     1.067983   
                  10    1.038328   0.985189         NaN     1.133934   

                           abate    abated  abatement      abbe    abbess  \
word_   period_   run_                                                      
station 1700-1705 1     0.982695  0.856937        NaN  1.132782  0.909438   
                  2     0.944321  0.632112   1.068780       NaN  0.867493   
                  3     0.933849  0.922525        NaN  1.071947  0.870604   
                  4     0.812107  0.920732   1.032329       NaN  0.892790   
                  5     0.941186  0.880589   1.070103  1.095534  0.972781   
...                          ...       ...        ...       ...       ...   
        1895-1900 6          NaN  0.784317        NaN  0.912622       NaN   
                  7     0.819167  0.899824        NaN  0.580467  0.755247   
                  8          NaN  0.840540   1.142362  0.859709  0.843648   
                  9          NaN  1.014848        NaN  0.980951  0.753701   
                  10         NaN  1.006114   0.757747  0.844385  0.850136   

                           abbey  ...   zealous    zenith    zephyr      zest  \
word_   period_   run_            ...                                           
station 1700-1705 1     1.042427  ...  0.885291       NaN       NaN       NaN   
                  2     0.935139  ...  0.998856       NaN       NaN       NaN   
                  3     0.966253  ...  0.860833       NaN       NaN       NaN   
                  4     1.228177  ...  1.036336       NaN       NaN       NaN   
                  5     1.080854  ...  1.077262       NaN       NaN       NaN   
...                          ...  ...       ...       ...       ...       ...   
        1895-1900 6     0.613744  ...  1.367641  0.936883  0.834988  1.091117   
                  7     0.538175  ...  1.164052  0.753413  1.215511  1.056076   
                  8     0.595659  ...  1.114407  0.836680  0.837813  1.104225   
                  9     0.637230  ...  1.114454  0.943961  1.073956  1.161853   
                  10    0.662767  ...  1.153319  0.870763  0.831703  0.921027   

                            zinc      zone       zoo  zoological   zoology  \
word_   period_   run_                                                       
station 1700-1705 1          NaN  0.871096  0.692374         NaN       NaN   
                  2          NaN  0.916769  0.790399         NaN       NaN   
                  3          NaN  0.983929  0.741790         NaN       NaN   
                  4          NaN  0.903570  0.658214         NaN       NaN   
                  5          NaN  0.890637  0.735945         NaN       NaN   
...                          ...       ...       ...         ...       ...   
        1895-1900 6     0.784255  0.806932  0.786050    0.829083  0.885881   
                  7     0.908829  0.761987  0.762617    0.745856  1.144429   
                  8     0.711716  0.872090  0.691038    0.719671  1.062151   
                  9     0.991612  0.739279  0.846314    0.700322  0.978648   
                  10    0.774325  0.821564  0.885272    0.801627  0.980594   

                 

In [5]:
# dfcdist.sort()

In [94]:

def lnm_neighborhoods(
        #dfnbr1,
        #dfnbr2,
        dfcdist1,
        dfcdist2,
        k=K,
        run_all_pairs=False,
        min_size=None,
        keep_runs=False,
        incl_words=False,
        **attrs):
    # filter words in both models
    s1=dfcdist1.mean().dropna()
    s2=dfcdist2.mean().dropna()
    valid_words_now=set(s1.index) & set(s2.index)
    s1=s1.loc[valid_words_now].sort_values()
    s2=s2.loc[valid_words_now].sort_values()
    
    # get top words for each
    nb1=s1.iloc[:k].index
    nb2=s2.iloc[:k].index
    
    # get meta neighborhoods
    mnb=list(set(nb1)|set(nb2))
    nb1s=s1.loc[mnb]
    nb2s=s2.loc[mnb]
    
    # try to get distance
    try:
        distdists = 1-fastdist.cosine(nb1s.values.astype(float), nb2s.values.astype(float))
    except ZeroDivisionError as e:
        distdists=np.nan
        
    # return dict as df
    odx={
        **attrs,
        'lnm':distdists,
        'mneighb_size':len(mnb),
        'neighb1_size':len(nb1),
        'neighb2_size':len(nb2),
        'neighb1':', '.join(nb1) if incl_words else '',
        'neighb2':', '.join(nb2) if incl_words else '',
    }
    odf=pd.DataFrame([odx])
    return odf

In [95]:
def _lnm_neighborhoods_(objd): return lnm_neighborhoods(**objd)


In [96]:
# lnm_word('culture',ymin=1700,ymax=1900,ybin=20)

In [97]:
def lnm_word(
        word,
        period_or_periods=None,
        run_or_runs=None,
        num_runs=10,
        word2=None,
        cache=True,
        force=False,
        k=K,
        cache_only=False,
        progress=True,
        progress_nbr=True,
        ymin=YMIN_DISTMAT,
        ymax=YMAX_NBR,
        ybin=YBIN_NBR,
        num_proc=1,
        incl_words=False):
    qstr=f'{word},ymin={ymin},ymax={ymax},ybin={ybin},k={k}'
    if cache and not force:
        with get_veclib('lnm') as vl:
            if qstr in vl: return vl[qstr] if not cache_only else pd.DataFrame()
    
    try:
        dfcdist = cdist_word(
            word,period_or_periods,run_or_runs,
            num_runs=num_runs,
            progress=progress_nbr,num_proc=num_proc,force=force,
            ymin=ymin,ymax=ymax,ybin=ybin
        ).loc[word]
    except KeyError:
        return pd.DataFrame()
    dfcdists_prds = list(dfcdist.groupby('period_'))
    objs = [
        dict(
            word=word,
            period1=prd1,period2=prd2,
#             dfnbr1=dfnbr1.loc[prd1],
#             dfnbr2=dfnbr2.loc[prd2],
            dfcdist1=dfprd1,
            dfcdist2=dfprd2,
            k=k,
            incl_words=incl_words,
        )
        for prd1,dfprd1 in dfcdists_prds
        for prd2,dfprd2 in dfcdists_prds
        if prd1<prd2
    ]
    o=pmap(
        _lnm_neighborhoods_,
        objs,
        num_proc=num_proc,
        progress=progress,
        desc='[Koselleck] Measuring LNM across period comparisons'
    )
    odf=pd.concat(o) if len(o) else pd.DataFrame()
    if cache:
        with get_veclib('lnm',autocommit=True) as vl:
            vl[qstr]=odf
    
    return odf if not cache_only else pd.DataFrame()

In [99]:
odf=lnm_word('virtue',num_proc=4,force=True,ybin=5,incl_words=True,k=10)
odf.iloc[0]

[Koselleck] (11:56:45) Finished running cdist_word(virtue) (+18.9s)
[Koselleck] Measuring LNM across period comparisons [x4]: 100%|██████████| 630/630 [00:37<00:00, 16.82it/s]


word                                                                                                          virtue
period1                                                                                                    1720-1725
period2                                                                                                    1725-1730
lnm                                                                                                        0.0456996
mneighb_size                                                                                                      20
neighb1_size                                                                                                      10
neighb2_size                                                                                                      10
neighb1         authority, integrity, popish, mankind, privileges, prejudice, sense, privilege, property, corruption
neighb2              misfortunes, benevolence, actions, absolute

In [101]:
odf.sort_values('lnm')

,word,period1,period2,lnm,mneighb_size,neighb1_size,neighb2_size,neighb1,neighb2
0,virtue,1855-1860,1860-1865,0.000360,12,10,10,"wisdom, spiritual, holiness, morality, moral, religion, piety, divine, faith, goodness","wisdom, spiritual, moral, piety, goodness, holiness, faith, humanity, creator, morality"
0,virtue,1885-1890,1890-1895,0.001471,12,10,10,"humanity, wisdom, faith, instinct, moral, spiritual, intellect, belief, morality, freedom","moral, humanity, wisdom, spiritual, instinct, belief, nature, intellect, religion, faith"
0,virtue,1865-1870,1880-1885,0.001597,13,10,10,"humanity, wisdom, spiritual, moral, intellect, morality, religion, sacrifice, piety, mankind","humanity, wisdom, spiritual, moral, faith, devotion, morality, divine, sacrifice, intellect"
0,virtue,1755-1760,1790-1795,0.001883,11,10,10,"virtuous, piety, wisdom, benevolence, social, goodness, folly, happiness, humanity, pride","virtuous, piety, wisdom, happiness, social, benevolence, humanity, mankind, folly, pride"
0,virtue,1755-1760,1805-1810,0.001931,11,10,10,"virtuous, piety, wisdom, benevolence, social, goodness, folly, happiness, humanity, pride","virtuous, piety, benevolence, wisdom, humanity, folly, social, mankind, happiness, pride"
...,...,...,...,...,...,...,...,...,...
0,virtue,1720-1725,1780-1785,0.138657,20,10,10,"authority, integrity, popish, mankind, privileges, prejudice, sense, privilege, property, corruption","virtuous, benevolence, ambition, pride, piety, innocence, virtues, wisdom, humanity, greatness"
0,virtue,1720-1725,1795-1800,0.140057,19,10,10,"authority, integrity, popish, mankind, privileges, prejudice, sense, privilege, property, corruption","virtuous, benevolence, piety, wisdom, mankind, humanity, social, rational, virtues, exalted"
0,virtue,1720-1725,1800-1805,0.142183,19,10,10,"authority, integrity, popish, mankind, privileges, prejudice, sense, privilege, property, corruption","wisdom, virtuous, benevolence, humanity, piety, social, mankind, virtues, generous, happiness"
0,virtue,1720-1725,1805-1810,0.143349,19,10,10,"authority, integrity, popish, mankind, privileges, prejudice, sense, privilege, property, corruption","virtuous, piety, benevolence, wisdom, humanity, folly, social, mankind, happiness, pride"


## lnm()

In [ ]:
def _lnm_(objd): return lnm_word(**objd)

def lnm(
        word_or_words,
        period_or_periods=None,
        run_or_runs=None,
        cache=True,
        force=False,
        cache_only=False,
        progress=True,
        progress_nbr=False,
        progress_word=None,
        num_proc=1,
        num_runs=NUM_RUNS_LNM,
        k=K,
        ymin=YMIN_NBR,
        ymax=YMAX_NBR,
        ybin=YBIN_NBR,
        ):
    words=tokenize_fast(word_or_words) if type(word_or_words)==str else list(word_or_words)
    
    objs=[
        dict(
            word=word,
            period_or_periods=period_or_periods,
            run_or_runs=run_or_runs,
            word2=None,
            cache=cache,
            force=force,
            k=k,
            cache_only=cache_only,
            progress=progress_word if progress_word is not None else (False if len(words)>1 else progress),
            progress_nbr=progress_nbr,
            num_proc=1 if len(words)>1 else num_proc,
            ymin=ymin,ymax=ymax,ybin=ybin,
            num_runs=num_runs
        ) for word in words
    ]
    o=pmap(
        _lnm_,
        objs,
        num_proc=num_proc if len(words)>1 else 1,
        progress=progress if len(words)>1 else False,
        desc='Measuring LNM across words',
    )
    return pd.concat(o) if len(o) else pd.DataFrame()


In [ ]:
# lnm('virtue,vice,virtues,vices,values')

In [ ]:
def lnm_precache_words(words=None,**y):
    words_done=set()
    with get_veclib('lnm') as vl:
        words_done=set(k.split(',')[0] for k in vl.keys())
    if not words: words=get_valid_words()
    words=[w for w in words if not w in words_done]
    lnm(words, cache_only=True,**y)

In [ ]:
# words=get_valid_words()
# random.shuffle(words)
# res=pmap(do_word, words, num_proc=1)

In [ ]:
# lnm_precache_words(get_all_nouns_adjs(), num_proc=3)

## Comparisons of magnitudes

In [ ]:
words=get_words_with_lnm()
len(words)

In [ ]:
def ttest_lnm_word(word,valkey='lnm',min_n=20):
    df=lnm_word(word)
    if not len(df): return pd.DataFrame()
    df=df.loc[word,word].reset_index()
    df.period1=df.period1.apply(lambda ystr: periodize_sattelzeit(int(ystr[:4]), use_dates=False))
    df.period2=df.period2.apply(lambda ystr: periodize_sattelzeit(int(ystr[:4]), use_dates=False))
    df['period_cmp']=[f'{x}-v-{y}' for x,y in zip(df.period1,df.period2)]
#     period_cmps={f'{x}-v-{y}' for x,y in zip(df.period1,df.period2) if x!=y}
    period_cmps={f'{x}-v-{y}' for x,y in zip(df.period1,df.period2) if x==y}
#     period_cmps={f'{x}-v-{y}' for x,y in zip(df.period1,df.period2)}
    o=[]
    for period_cmp in period_cmps:
        g=df[df.period_cmp==period_cmp]
        p1,p2=period_cmp.split('-v-')
#         gnull=df[df.period_cmp.isin({f'{p1}-v-{p1}', f'{p2}-v-{p2}'})]
        gnull=df[(df.period_cmp!=period_cmp) & df.period_cmp.isin(period_cmps)]
        a=gnull[valkey]
        b=g[valkey]
        if len(a)<min_n or len(b)<min_n: continue
        mw,mw_p=mannwhitneyu(a,b)
        o+=[dict(
            word=word,
            vector='LNM',
            period_cmp=period_cmp,
            n1=len(a),
            n2=len(b),
            mw=mw,
            mw_p=mw_p,
            avg1=a.mean(),
            avg2=b.mean(),
        )]
    df=pd.DataFrame(o)
#     if len(df):
#         df['avg_diff']=df.avg2 - df.avg1
#         df['avg_div']=df.avg2/df.avg1
    return df

In [ ]:
ttest_lnm_word('sympathy')

In [ ]:
def ttest_lnm_words(force=False,num_proc=1):
    if not force and os.path.exists(FN_LNM_TTEST): return read_df(FN_LNM_TTEST)
    
    words=get_words_with_lnm()
    o=pmap(
        ttest_lnm_word,
        words,
        num_proc=num_proc
    )
    if not len(o): return pd.DataFrame()
    odf=pd.concat(o) if len(o) else pd.DataFrame()
    odf['mw_perc']=odf.mw.rank(ascending=False) / len(odf) * 100
    odf=odf.set_index(['word','vector','period_cmp']).sort_values('mw')
    odf=pd.concat(
        vdf.assign(mw_perc_vec=vdf.mw.rank(ascending=False) / len(vdf) * 100)
        for i,vdf in odf.groupby('vector')
    )
    odf=pd.concat(
        vdf.assign(mw_perc_vec_cmp=vdf.mw.rank(ascending=False) / len(vdf) * 100)
        for i,vdf in odf.groupby(['vector','period_cmp'])
    )
    odf=pd.concat(
        vdf.assign(
            avg1_perc_vec=(vdf.avg1.rank(ascending=False) / len(vdf) * 100),
            avg2_perc_vec=(vdf.avg2.rank(ascending=False) / len(vdf) * 100),
        )
        for i,vdf in odf.groupby('vector')
    )
    
    odf=odf.sort_index()
    s=odf.avg1.append(odf.avg2)
    odf.avg1 = (odf.avg1 - s.mean())/s.std()
    odf.avg2 = (odf.avg2 - s.mean())/s.std()
    odf['avg_diff']=odf.avg2 - odf.avg1
    odf['avg_div']=odf.avg2/odf.avg1
    
    odf.to_pickle(FN_LNM_TTEST)
    
    # add to db
    with get_veclib('ttest_lnm') as vl:
        for w,wdf in tqdm(odf.groupby('word'),desc='Adding to db'):
            vl[w]=wdf
        print('Committing')
        vl.commit()
        print('Done')
    
    return odf

In [ ]:
odf=ttest_lnm_words(num_proc=4,force=True)
odf

In [ ]:
odf.query('period_cmp=="Before-v-After"').groupby('word').mean().sort_values('mw',ascending=True).head(25)